# Dimensionality reduction: PCA (Principal Component Analysis)

Key take-aways
- PCA is highly sensitive to data scaling
- It finds a basis of orthogonal vectors for the input data that are uncorrelated with each other
- The orthogonal don't have to be independent however!


The basic idea of PCA is to reduce the dimensionality of vectors $\pmb{x}\in\mathbb{R}^D$ to a low-dimensional latent space $\pmb{z}\in\mathbb{R}^L$, through a linear orthogonal projection on the subspace that keeps the most variance of $\pmb{x}$. For an input dataset of $N$ examples, we construct the design matrix $X\in\mathbb{R}^{N\times D}$. Intuitively one can already understand that there might be a lot of redundant information in this matrix when there are correlated variables. PCA will extract these *linear* relationships and reduce the dimensionality of the data. 

One way to derive the (inverse) projection matrix $W\in\mathbb{R}^{D\times L}$ is to minimize the reconstruction loss: 
$$ \mathcal{L}(W, Z) = \frac{1}{N}\sum_{n=1}^N\|\pmb{x}_n-W\pmb{z}_n\|^2$$
where we are approximating each $\pmb{x}_n$ by $\hat{\pmb{x}}_n=W\pmb{z}_n$. W then consists of the $L$ principal components: $W = \begin{bmatrix}\pmb{w}_1 & ... & \pmb{w}_L \end{bmatrix}$ and $\pmb{z}_n$ are the *scores* for example $n$.

In a first step, let us start by estimating the best 1d solution $\pmb{w}_1\in\mathbb{R}^D$ and its associated scores $\pmb{z}_1\in\mathbb{R}^N$: 
$$\mathcal{L}(\pmb{w}_1, \pmb{z}_1) = \frac{1}{N}\sum_{i=1}^N (\pmb{x}_i-z_{i1}\pmb{w}_1)^T(\pmb{x}_i-z_{i1}\pmb{w}_1) = \frac{1}{N}\sum_{i=1}^N \pmb{x}_i^T\pmb{x}_i -2z_{i1}\pmb{w}_1^T\pmb{x}_i + z_{i1}^2\pmb{w}_1^T\pmb{w}_1$$

Here $\pmb{w}_1^T\pmb{w}_1 = 1$ because of the orthonormality condition, and we can further calculate the values of $z_{i1}$ for which $\mathcal{L}$ is minimized: 
$$ \frac{\partial}{\partial z_{in}}\mathcal{L}(\pmb{w}_1, \pmb{z}_1) = -2\pmb{w}_1^T\pmb{x}_i + 2*z_{i1} \rightarrow z_{i1} = \pmb{w}_1^T\pmb{x}_i$$

Thus the optimal embedding is obtained by orthogonally projecting the data onto $\pmb{w}_1$. Substituting this in the loss function, which is now only dependent on $\pmb{w}_1$: 
$$\mathcal{L}(\pmb{w}_1) = \frac{1}{N}\sum_{i=1}^N \pmb{x}_i^T\pmb{x}_i -2(\pmb{w}_1^T\pmb{x}_i)(\pmb{w}_1^T\pmb{x}_i) + (\pmb{w}_1^T\pmb{x}_i)^2 = \frac{1}{N}\sum_{i=1}^N \pmb{x}_i^T\pmb{x}_i - (\pmb{w}_1^T\pmb{x}_i)^2 = \text{const} - \pmb{w}_1^T\pmb{x} \pmb{x}^T\pmb{w}_1 = \text{const} - \pmb{w}_1^T\hat{\Sigma} \pmb{w}_1$$

The lagrangian subject to the constraint that $\pmb{w}^T\pmb{w}=1$ then becomes: 
$$ \tilde{\mathcal{L}}(\pmb{w}_1) = \pmb{w}_1^T\hat{\Sigma} \pmb{w}_1 + \lambda_1(\pmb{w}_1^T\pmb{w}_1-1) $$

This lagrangian is maximized when 
$$\frac{\partial}{\partial \pmb{w}_1}\tilde{\mathcal{L}}(\pmb{w_1}) = 2\hat{\Sigma}\pmb{w}_1 -2\lambda_1\pmb{w}_1=0 \rightarrow \hat{\Sigma}\pmb{w}_1 = \lambda_1\pmb{w}_1$$


Therefore $\pmb{w}_1$ is the eigenvector corresponding to the largest eigenvalue of the empirical covariance matrix $\Sigma$

### Part 2: Probabilistic interpretation

PCA can be interpreted in a probabilistic setting through the lens of Factor Analysis. In such a setting, it is assumed that $\pmb{x}$ is a linear transformation of a normally distributed random variable $\pmb{z}$ of lower dimensionality: $p(\pmb{z}) = \mathcal{N}(\pmb{z}|\pmb{0},\pmb{I})$. The conditional model is then: $p(\pmb{x}|\pmb{z}) = \mathcal{N}(\pmb{x}|W\pmb{x}+\pmb{\mu},\pmb{\Psi})$. The model for $\pmb{x}$ then becomes: 
$$ p(\pmb{x}) = \int_{\pmb{z}}p(\pmb{x}|\pmb{z})p(\pmb{z})d\pmb{z} = \mathcal{N}(\pmb{x}|\pmb{\mu}, WW^T+\Psi)$$

For PCA, W is enforced to be orthonormal such that $\Psi = \sigma^2\mathbb{I}$. The log-likelihood of the data under this distribution can be calculated and the MLE of the parameters $\sigma$ and $W$ can then be found. For $\sigma\rightarrow 0$, this solution collapses to the regular PCA solution. 

### Part 3: Implementation

Notes on the implementation:
- eig returns the eigenvectors as vectors next to each other: $[v_1 ... v_k]$, which are the principal components
- the scikit-learn PCA class stores the components row-wise however
- The explained_variance_ are just the eigenvalues of the covariance matrix
- It DOES matter that the covariance matrix is calculated by deviding by (n-1) instead of n
- You can just multiply on the other side to go from x to z: W^T * (W*x) = W^T * z = x since W^T W = I

In [91]:
import numpy as np 
from scipy.linalg import svd, eig

class myPCA:
    """ My own implementation of PCA, which follows the scikit-learn API"""
    def __init__(self, n_components=None, whiten=False) -> None:
        self.n_components = n_components
        self.whiten = whiten
    def fit(self, X: np.ndarray):
        assert len(X.shape) == 2
        self.n_samples_, self.n_features_ = X.shape
        self.mean_ = X.mean(axis=0)
        Xc = X - self.mean_ 
        S = Xc.T @ Xc / (self.n_samples_-1)
        S = np.cov(X.T)
        _, _, Vt = svd(Xc, full_matrices=False)
        assert S.shape == (self.n_features_, self.n_features_)
        lambdas, vs = eig(S)
        sorting = np.argsort(lambdas)[::-1]
        self.explained_variance_ = lambdas[sorting]
        self.explained_variance_ratio_ = self.explained_variance_ / sum(self.explained_variance_)
        self.components_ = vs[:, sorting].T  # The scikit-learn API puts the components in rows!
        

    def get_covariance(self):
        """Compute data covariance with the generative model."""
        pass 
    def transform(self, X):
        """Apply dimensionality reduction to X."""
        return (X-self.mean_) @ self.components_.T  # z = W^T * x
    def inverse_transform(self, Z):
        """Transform data back to original space."""
        return self.components_ @ Z  # x=W*z 
    def score(self, X):
        """Return the average log-likelihood of all samples."""
        pass 
    def score_samples(self, X):
        """Return the log likelihood of each sample."""
        pass 

In [92]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
wine = datasets.load_wine()
wine_df = pd.DataFrame(data=wine.data, columns=wine['feature_names']) 
wine_df['target'] = pd.Series(wine['target_names'][wine['target']], dtype="category")
wine_df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,class_0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(wine_df.drop('target', axis=1).values, wine_df['target'].values)

In [93]:
from sklearn.decomposition import PCA 
skpca = PCA(n_components=3)
mypca = myPCA()
skpca.fit(X_train)
mypca.fit(X_train)
print("The first principal component: ")
print(skpca.components_[0,:])
print(mypca.components_[0,:])


The first principal component: 
[ 1.64674979e-03 -6.68466219e-04  1.85354140e-04 -5.12219565e-03
  1.84956010e-02  1.01414919e-03  1.47163437e-03 -1.34844500e-04
  5.33631358e-04  2.31938304e-03  1.50302509e-04  7.17936506e-04
  9.99809516e-01]
[-1.64674979e-03  6.68466219e-04 -1.85354140e-04  5.12219565e-03
 -1.84956010e-02 -1.01414919e-03 -1.47163437e-03  1.34844500e-04
 -5.33631358e-04 -2.31938304e-03 -1.50302509e-04 -7.17936506e-04
 -9.99809516e-01]


In [94]:
print(skpca.transform(X_test)[:,1])
print(mypca.transform(X_test)[:,1])

[  9.47820042   6.4271982    1.29776714  -0.73965734  -6.00940331
  -6.80326595 -20.41467372  -6.83644349   4.60658186 -15.51428216
  59.50300987   0.59737902  -0.18589483  23.40864747   3.87981494
 -14.36678885   3.39020368  19.76266928  28.63978721  -2.57933646
  26.39802441   6.25607049  23.83262819  -5.15350197   2.52981539
  -8.57129744 -12.9542068   -0.6057908    9.22987363  -0.35615751
  12.03944015  -8.09515335  10.41818156  -9.2648081   -1.93327944
   4.84677119  -7.50184558   5.31686501  -7.68614472  -9.46455766
   6.07103073  -3.80909118   5.47667016  -9.2292848   52.43190883]
[ -9.47820042  -6.4271982   -1.29776714   0.73965734   6.00940331
   6.80326595  20.41467372   6.83644349  -4.60658186  15.51428216
 -59.50300987  -0.59737902   0.18589483 -23.40864747  -3.87981494
  14.36678885  -3.39020368 -19.76266928 -28.63978721   2.57933646
 -26.39802441  -6.25607049 -23.83262819   5.15350197  -2.52981539
   8.57129744  12.9542068    0.6057908   -9.22987363   0.35615751
 -12.0394

In [95]:
# The explained variance doesn't change when a different number of components is used
print(skpca.explained_variance_ratio_)
print(mypca.explained_variance_ratio_.real)

[9.98292699e-01 1.52483167e-03 1.01618221e-04]
[9.98292699e-01 1.52483167e-03 1.01618221e-04 5.18861180e-05
 1.24410004e-05 9.46696498e-06 3.00790091e-06 1.42816676e-06
 1.11531559e-06 7.62138779e-07 4.08724879e-07 2.46288634e-07
 8.84364932e-08]


In [68]:
test = np.random.randn(3,4)
print(test)
print(test[:, [2, 1, 0]])

[[-0.97881657 -0.05072107  0.42971562  0.30813218]
 [-1.49511106  1.19490758 -0.71988722  0.64025099]
 [-0.76254381  0.74116086 -1.10593211 -0.50357508]]
[[ 0.42971562 -0.05072107 -0.97881657]
 [-0.71988722  1.19490758 -1.49511106]
 [-1.10593211  0.74116086 -0.76254381]]
